# Prepairing chesscom data to use

Working with API, prepairing good forma etc

## Imports and functions

In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime
import json

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# for file working
from os import listdir
from os.path import isfile, join

In [2]:
# # for logging our process
# def logging(proc, par):
#     import datetime
#     now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

#     import json
#     params_dict={}
#     for val in par:
#         params_dict.update(val)
        
#     params = json.dumps(params_dict)

#     log_df=pd.DataFrame([now, proc, params]).T
#     log_df.columns=['time', 'proc', 'params']

#     log_df.to_csv('log.csv', index=False, header=False, mode='a+')

In [3]:
# heplfull instruments

# convert unixtime to time:
# fr_unixtime(1565211491) -> '2019-08-07 20:58:11'
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# convert integer month number to string API format:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
# 112 -> -1
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1
    
# retunr classoc Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    return 1/(1+np.power(10, (rb-rw)/400))

# working with pgn
# pip install pgn-parser
def get_pgn(text):
    from pgn_parser import pgn, parser
    game = parser.parse(text, actions=pgn.Actions())
    
    try:
        score=game.tag_pairs['Result']
    except:
        score='Unknown'  
    try:
        date=game.tag_pairs['Date']
    except:
        date='Unknown'
    try:
        time=game.tag_pairs['UTCTime']
    except:
        time='Unknown'
    try:
        eco=game.tag_pairs['ECO']
    except:
        eco='Unknown'
    try:
        ecourl=game.tag_pairs['ECOUrl']
    except:
        ecourl='Unknown'
    return {'score':score, 
            'date': date, 
            'time':time, 
            'ECO':eco, 
            'ECO_url':ecourl}

# functions for predictions
# find col with target user (white or black)
def find_col(target_user, col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
# reverse fun of find_col    
def find_opp(target_user, col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val

In [4]:
# get API data and prepairint it

# get json form site and return string
# exmple get_api_data_to_str('eric', '2014', '01')
def get_api_data_to_str(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    
#     if response.status_code ==200:
#         proc='api_call_sucess'
#         logging(proc, 
#                 [
#                     {'player':player},
#                     {'year': year},
#                     {'month': month},
#                     {'type': 'norm'}
#                 ]
#                )
#     if response.status_code !=200:
#         proc='api_call_error'
#         logging(proc, 
#                 [
#                     {'player':player},
#                     {'year': year},
#                     {'month': month},
#                     {'type': 'error'},
#                     {'error_code': response.status_code},
#                     {'error_message': st},
#                 ]
#                )
    return st, response.status_code

# convert api data to pandas for next working
def get_str_data_to_pandas(strng):
    import json
    try:
        js = json.loads(strng)
        df=pd.DataFrame(js)
    except:
        proc='data_parse_error'
#         logging(proc, 
#                 [
#                     {'type': 'error'},
#                     {'source': strng[0:3000]}
#                 ]
#                )       
    return df

# convert multilevel json to table
def prepair_pandas_multilevel_data(df):
    num=len(df)
    new_df=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(df.values[i])
        new_df=pd.concat([new_df, b])
    return new_df 

# get player games in month with preparation
def get_pl_stat(game_stat):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    # number of games
    num=len(game_stat)
    
    # cut long and unusefull columns
    game_stat=game_stat[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    # score 0.0, 0.5 or 1.0
    game_stat['white.score']=game_stat['white.result'].map(res_dict)
    game_stat['black.score']=game_stat['black.result'].map(res_dict)
    
    # pgn - long string
    game_stat['pgn']=game_stat['pgn'].apply(get_pgn)
    # result - string like '1-0'
    game_stat['result']=game_stat['pgn'].apply(lambda x: x['score'])
    
    game_stat['date']=game_stat['pgn'].apply(lambda x: x['date'])
    game_stat['time']=game_stat['pgn'].apply(lambda x: x['time'])
    
    # ECO - string like 'B10'
    game_stat['eco']=game_stat['pgn'].apply(lambda x: x['ECO'])
    # ECOurl - url like 'https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5'
    game_stat['eco_url']=game_stat['pgn'].apply(lambda x: x['ECO_url'])
    
    # propability from classic formula
    game_stat['white_elo_forecast']=game_stat[['white.rating', 'black.rating']].apply(lambda x: elo_prob(*x), axis=1)
    game_stat['black_elo_forecast']=game_stat[['black.rating', 'white.rating']].apply(lambda x: elo_prob(*x), axis=1)
    
    # after it np.sum(df['game']) means number of games
    game_stat['game']=1
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    # period - stirng like '2019-08'
    game_stat['period']=game_stat['date'].dt.year.astype('str')+'-'+game_stat['date'].dt.month.astype('str')
        
    game_stat.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'date', 'time', 'eco',
       'eco_url', 'white_elo_forecast', 'black_elo_forecast',
       'game', 'period']
    game_stat=game_stat[[
        'rules', 'time_class', 'time_control', 'rated', 'game',
        'result', 'date', 'time', 'period',
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'white_elo_forecast', 'black_elo_forecast',
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'
    ]]
    
    
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
#     proc='pl_stat'
    player=game_stat['white.username'].value_counts().index[0]
    year=game_stat['date'].value_counts().index[0].strftime("%Y")
    month=game_stat['date'].value_counts().index[0].strftime("%m")
#     logging(proc, 
#         [
#             {'player':player},
#             {'year': year},
#             {'month': month},
#             {'type': 'norm'}
#         ]
#        )
    return game_stat

# data for predict current user
def learn_prepair(df, target_user):
    df=df[(df['white.username']==target_user)|(df['black.username']==target_user)]
    df['target_user']=target_user
    df['score']=df[['target_user', 'white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['target_user','white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['target_user', 'white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['target_user', 'white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    
    df['date'] = df['date'].astype('datetime64[ns]')
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    
    return df
    


In [14]:
# def status(player, year, month, status, is_error):
#     import datetime
#     now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#     buf_df=pd.DataFrame([[player, year, month, now, status, is_error]], 
#                         columns=['player', 'year', 'month', 'time','status', 'is_error'])
#     buf_df.to_csv('data/status.csv', index=False, header=False, mode='a+')    

In [15]:
# # main function
# def get_btch(player, year, month):
#     import datetime
#     import time
#     import json

#     status(player, year, month, 'new', is_error=0)
    
    
#     # list random seconds of delay
#     delay_lst=[np.round(1*np.random.rand(1,1)[0,0],0), 
#                np.round(5*np.random.rand(1,1)[0,0],0), 
#                np.round(100*np.random.rand(1,1)[0,0],0), ]
    
#     # we give 3 attemps to get data
#     for cs in range(3):    
#         time.sleep(delay_lst[cs])
#         # working with propably exception
#         try:
#             # API -> string
#             data_string, code=get_api_data_to_str(player, year, month)
#         except:
#             data_string=''
#             code=500
#         # after it we need to check status code
#         # case best variant
#         if code==200:
#             proc='api_call_saved'
#             logging(proc, 
#                     [
#                         {'player':player},
#                         {'year': year},
#                         {'month': month},
#                         {'type': 'norm'}
#                     ]
#                    )
#             # exit from cicle 
#             break
#         # else log and try again
#         else:
#             api_error_str='api error, code: '+str(code)+', message: '+data_string[0:3000]+'case: '+str(cs)
#             status(player, year, month, api_error_str, is_error=1)
#             proc='api_call_error_recieved'
#             logging(proc, 
#                     [
#                         {'player':player},
#                         {'year': year},
#                         {'month': month},
#                         {'type': 'error'},
#                         {'try':cs},
#                         {'error_code': code},
#                         {'error_message': data_string[0:3000]}
#                     ]
#                    ) 
#         # at the end we need to log the end of our attepts
#         if cs==2:
#             proc='api_call_final_error'
#             logging(proc, 
#                     [
#                         {'player':player},
#                         {'year': year},
#                         {'month': month},
#                         {'type': 'error'},
#                         {'try':cs},
#                         {'error_code': code},
#                         {'error_message': data_string[0:3000]}
#                     ]
#                    )
    
#     # string -> json -> DataFrame
#     try:
#         multidata=get_str_data_to_pandas(data_string)
#     except:
#         status(player, year, month, 'error_to_pandas', is_error=1)
    
#     # DataFrame -> use json normalise -> good df
#     try:
#         usefull_data=prepair_pandas_multilevel_data(multidata)
#     except:
#         status(player, year, month, 'multilevel error', is_error=1)
    
#     # game prepair
#     try:
#         workng_data=get_pl_stat(usefull_data)
#     except:
#         status(player, year, month, 'coverting error', is_error=1)
    
#     # save to file
#     try:
#         now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#         workng_data.to_csv('source/'+player+'_'+year+'_'+month+'_'+now+'.csv')
#         suc_str='successfull get data: '+str(len(workng_data))+' rows'
#         status(player, year, month, suc_str, is_error=0)
#     except:
#         status(player, year, month, 'writing error', is_error=1)
        
#     return multidata

In [16]:
# # start getting with threading magic
# def user_parse(user_list, year_range):   
#     import threading
#     import json
    
#     i=0
#     for user in user_list:
#         for year in year_range:
#             for month in range(1,13):
#                 t = threading.Thread(target=get_btch, args=(user, str(year), get_number(month)))
#                 t.start()
                

#                 i=i+1
#     print('All btch', i,  'are started')   


In [5]:
# save data butch from API to file and create navigation file
def save_data(player, year, month):
    # main API function
    data_string, code=get_api_data_to_str(player, year, month)
    
    filenamepath='data/'+str(player)+'_'+str(year)+'_'+str(month)+'.json'
    
    # write main data file
    with open(filenamepath, 'w') as outfile:
        json.dump(data_string, outfile)
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # write status file
    status_string=str(player)+';'+str(year)+';'+str(month)+';'+str(now)+';'+str(code)+';'+filenamepath+';'+data_string[0:180]
    text_file = open('log/'+str(player)+'_'+str(year)+'_'+str(month)+'.csv', 'w')
    text_file.write(status_string)
    text_file.close()

In [6]:
# read all json in folder and concatenate DataFrames
def read_files(path):
    # reading
    l=[]
    cntr=0
    import os
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".csv"):
                 l.append(os.path.join(root, file))
            cntr=cntr+1
    print('Total', cntr, 'files founded')

    # merging
    i=0
    df=pd.DataFrame()
    for link in l:
        try:
            dfb=pd.read_csv(link)
        except:
            dfb=pd.DataFrame()
        df=pd.concat([df, dfb])
        
    df=df.drop_duplicates()
    return df

In [7]:
# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
    try:
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write(st)
        file.close()
    except Exception:
        error_st=response.status_code+st[0:3000]
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write( error_st)
        file.close()


In [8]:
# load fata from file and convert it to usefull DataFrame
def load_games(file):
    with open(file) as json_file:
        jsf = json.load(json_file)
    jss = json.loads(jsf)
    first_df=pd.DataFrame(jss)
    
    game_stat=prepair_pandas_multilevel_data(first_df)
    res=get_pl_stat(game_stat)
    
    return res

In [51]:
# the function to launch
def get_data(user_list, year_range):
    # user cicle
    for user in user_list:
        # status to visulise
        print('Loading of games from user ', user, ' started')
        for year in year_range:
            print('Year ', year, ' is beginning')
            for month in range(1,13):
                now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(now, ' : month ', month, ' started')
                try:
                    save_data(user,str(year), get_number(month)) 
                except:
                    print('Error')
                    # write error string
                    error_string=str(user)+';'+str(year)+';'+str(month)+';'+str(now)
                    text_file = open('error/'+str(user)+'_'+str(year)+'_'+str(month)+'.csv', 'w')
                    text_file.write(error_string)
                    text_file.close()
                    

## Start working

In [10]:
%%time
# user_list=['Rosolimo', 'Akash22', 'smcwhtdtmc']
user_list=['Rosolimo']
year_range=range(2018,2020)

get_data(user_list, year_range)

Loading of games from user  Rosolimo  started
Year  2018  is beginning
2020-01-18 12:09:20  : month  1  started
2020-01-18 12:09:22  : month  2  started
2020-01-18 12:09:23  : month  3  started
2020-01-18 12:09:26  : month  4  started
2020-01-18 12:09:29  : month  5  started
2020-01-18 12:09:33  : month  6  started
2020-01-18 12:09:38  : month  7  started
2020-01-18 12:09:41  : month  8  started
2020-01-18 12:09:44  : month  9  started
2020-01-18 12:09:51  : month  10  started
2020-01-18 12:09:54  : month  11  started
2020-01-18 12:09:59  : month  12  started
Year  2019  is beginning
2020-01-18 12:10:06  : month  1  started
2020-01-18 12:10:11  : month  2  started
2020-01-18 12:10:14  : month  3  started
2020-01-18 12:10:19  : month  4  started
2020-01-18 12:10:22  : month  5  started
2020-01-18 12:10:24  : month  6  started
2020-01-18 12:10:25  : month  7  started
2020-01-18 12:10:27  : month  8  started
2020-01-18 12:10:30  : month  9  started
2020-01-18 12:10:33  : month  10  starte

In [42]:
mypath='log/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

log_df=pd.DataFrame()
b=pd.DataFrame()
for file in onlyfiles:
    b=pd.read_csv('log/'+file, header=None, sep=';')
    log_df=pd.concat([log_df, b])
    
log_df.columns=['user', 'year', 'month', 'last_update_time', 'code', 'file', 'caption']

In [46]:
log_df.sample(4)

,index,user,year,month,last_update_time,code,file,caption,is_games,buf
6912,0,Bragu78,2018,1,2020-01-18 12:56:41,200,data/Bragu78_2018_01.json,"{""games"":[{""url"":""https://www.chess.com/live/game/2524243054"",""pgn"":""[Event \""Live Chess\""]\n[Site \""Chess.com\""]\n[Date \""2018.01.01\""]\n[Round \""-\""]\n[White \""nour-uv\""]\n[Black",1,6912
8929,0,Sylvania88,2016,4,2020-01-18 16:56:21,200,data/Sylvania88_2016_04.json,"{""games"":[]}",0,8929
4805,0,Steinstone,2018,6,2020-01-18 17:02:00,200,data/Steinstone_2018_06.json,"{""games"":[{""url"":""https://www.chess.com/live/game/2883461095"",""pgn"":""[Event \""Live Chess\""]\n[Site \""Chess.com\""]\n[Date \""2018.06.17\""]\n[Round \""-\""]\n[White \""Steinstone\""]\n[Bl",1,4805
2625,0,kareemdagher,2018,8,2020-01-18 16:21:56,200,data/kareemdagher_2018_08.json,"{""games"":[{""url"":""https://www.chess.com/live/game/2976974757"",""pgn"":""[Event \""Live Chess\""]\n[Site \""Chess.com\""]\n[Date \""2018.08.01\""]\n[Round \""-\""]\n[White \""timothyha\""]\n[Bla",1,2625


In [44]:
log_df['is_games']=np.where(log_df['caption']=='{"games":[]}',0,1)
log_df=log_df.reset_index()
log_df['buf']=log_df.index
log_df.groupby(['code', 'is_games']).agg({'buf': lambda x: x.nunique()}).reset_index()

,code,is_games,buf
0,200,0,4595
1,200,1,3841
2,404,1,1089


In [28]:
log_df[log_df['user']=='Rosolimo'].sort_values(by=['year','month'])[0:2]

,index,user,year,month,last_update_time,code,file,caption,is_games,buf
12,0,Rosolimo,2018,1,2020-01-18 12:09:22,200,data/Rosolimo_2018_01.json,"{""games"":[{""url"":""https://www.chess.com/live/game/2524167256"",""pgn"":""[Event \""Live Chess\""]\n[Site \""Chess.com\""]\n[Date \""2018.01.01\""]\n[Round \""-\""]\n[White \""Rosolimo\""]\n[Blac",1,12
15,0,Rosolimo,2018,2,2020-01-18 12:09:23,200,data/Rosolimo_2018_02.json,"{""games"":[{""url"":""https://www.chess.com/live/game/2588427174"",""pgn"":""[Event \""Live Chess\""]\n[Site \""Chess.com\""]\n[Date \""2018.02.01\""]\n[Round \""-\""]\n[White \""Rosolimo\""]\n[Blac",1,15


In [19]:
files_lst=log_df[log_df['user']=='Rosolimo']['file']

In [21]:
tst_df=pd.DataFrame()
for file in files_lst:
    tst_df=pd.concat([tst_df, load_games(file)])

/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
len(tst_df)

2843

In [23]:
tst_df.sample(3)

,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url
0,chess,blitz,180+2,True,1,1-0,2018-04-18,21:06:10,2018-4,Rosolimo,jsalimjr,1613,1586,win,resigned,1.0,0.0,0.538778,0.461222,A40,https://www.chess.com/openings/Queens-Pawn-Opening-English-Defense,https://www.chess.com/live/game/2754297874,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/jsalimjr
0,chess,blitz,180+2,True,1,0-1,2019-04-19,12:31:25,2019-4,hiteshissar,Rosolimo,1527,1584,timeout,win,0.0,1.0,0.418699,0.581301,A06,https://www.chess.com/openings/Reti-Opening-Nimzowitsch-Larsen-Attack-2...c5,https://www.chess.com/live/game/3629866955,https://api.chess.com/pub/player/hiteshissar,https://api.chess.com/pub/player/rosolimo
0,chess,blitz,180+2,True,1,1-0,2018-01-17,09:03:28,2018-1,Rosolimo,sourabhsri,1599,1517,win,abandoned,1.0,0.0,0.615864,0.384136,D35,https://www.chess.com/openings/Queens-Gambit-Declined-Queens-Knight-Variation-3...Nf6,https://www.chess.com/live/game/2557042323,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/sourabhsri


In [24]:
opponent_lst=tst_df[['white.username']].drop_duplicates().sample(100)

In [33]:
len(opponent_lst)

100

In [36]:
opponent_lst['white.username'].values[0:5]

array(['amlanmodak', 'abhishekshanu', 'sergey90', 'Vasylii', 'JAVERT1971'],
      dtype=object)

In [38]:
%%time
year_range=range(2013,2021)

get_data(opponent_lst['white.username'][5:].values, year_range)

Loading of games from user  Sev30  started
Year  2013  is beginning
2020-01-18 12:52:02  : month  1  started
2020-01-18 12:52:03  : month  2  started
2020-01-18 12:52:04  : month  3  started
2020-01-18 12:52:04  : month  4  started
2020-01-18 12:52:06  : month  5  started
2020-01-18 12:52:07  : month  6  started
2020-01-18 12:52:08  : month  7  started
2020-01-18 12:52:09  : month  8  started
2020-01-18 12:52:10  : month  9  started
2020-01-18 12:52:11  : month  10  started
2020-01-18 12:52:11  : month  11  started
2020-01-18 12:52:12  : month  12  started
Year  2014  is beginning
2020-01-18 12:52:13  : month  1  started
2020-01-18 12:52:14  : month  2  started
2020-01-18 12:52:15  : month  3  started
2020-01-18 12:52:16  : month  4  started
2020-01-18 12:52:17  : month  5  started
2020-01-18 12:52:18  : month  6  started
2020-01-18 12:52:19  : month  7  started
2020-01-18 12:52:20  : month  8  started
2020-01-18 12:52:21  : month  9  started
2020-01-18 12:52:22  : month  10  started
2

2020-01-18 12:58:35  : month  8  started
2020-01-18 12:58:36  : month  9  started
2020-01-18 12:58:37  : month  10  started
2020-01-18 12:58:38  : month  11  started
2020-01-18 12:58:39  : month  12  started
Loading of games from user  conmar  started
Year  2013  is beginning
2020-01-18 12:58:39  : month  1  started
2020-01-18 12:59:01  : month  2  started
2020-01-18 12:59:15  : month  3  started
2020-01-18 12:59:33  : month  4  started
2020-01-18 12:59:51  : month  5  started
2020-01-18 13:00:11  : month  6  started
2020-01-18 13:00:35  : month  7  started
2020-01-18 13:00:58  : month  8  started
2020-01-18 13:01:14  : month  9  started
2020-01-18 13:01:36  : month  10  started
2020-01-18 13:01:52  : month  11  started
2020-01-18 13:02:22  : month  12  started
Year  2014  is beginning
2020-01-18 13:02:53  : month  1  started
2020-01-18 13:03:14  : month  2  started
2020-01-18 13:03:32  : month  3  started
2020-01-18 13:03:48  : month  4  started
2020-01-18 13:04:06  : month  5  starte

2020-01-18 13:26:50  : month  3  started
2020-01-18 13:26:50  : month  4  started
2020-01-18 13:26:51  : month  5  started
2020-01-18 13:26:52  : month  6  started
2020-01-18 13:26:53  : month  7  started
2020-01-18 13:26:54  : month  8  started
2020-01-18 13:26:54  : month  9  started
2020-01-18 13:26:56  : month  10  started
2020-01-18 13:26:57  : month  11  started
2020-01-18 13:26:58  : month  12  started
Loading of games from user  iNAIMi  started
Year  2013  is beginning
2020-01-18 13:26:59  : month  1  started
2020-01-18 13:26:59  : month  2  started
2020-01-18 13:27:01  : month  3  started
2020-01-18 13:27:02  : month  4  started
2020-01-18 13:27:03  : month  5  started
2020-01-18 13:27:06  : month  6  started
2020-01-18 13:27:11  : month  7  started
2020-01-18 13:27:15  : month  8  started
2020-01-18 13:27:17  : month  9  started
2020-01-18 13:27:20  : month  10  started
2020-01-18 13:27:20  : month  11  started
2020-01-18 13:27:22  : month  12  started
Year  2014  is beginnin

2020-01-18 13:35:36  : month  11  started
2020-01-18 13:35:39  : month  12  started
Year  2020  is beginning
2020-01-18 13:35:43  : month  1  started
2020-01-18 13:35:46  : month  2  started
2020-01-18 13:35:47  : month  3  started
2020-01-18 13:35:48  : month  4  started
2020-01-18 13:35:49  : month  5  started
2020-01-18 13:35:50  : month  6  started
2020-01-18 13:35:51  : month  7  started
2020-01-18 13:35:52  : month  8  started
2020-01-18 13:35:52  : month  9  started
2020-01-18 13:35:53  : month  10  started
2020-01-18 13:35:54  : month  11  started
2020-01-18 13:35:55  : month  12  started
Loading of games from user  vitas123  started
Year  2013  is beginning
2020-01-18 13:35:55  : month  1  started
2020-01-18 13:35:56  : month  2  started
2020-01-18 13:35:56  : month  3  started
2020-01-18 13:35:56  : month  4  started
2020-01-18 13:35:57  : month  5  started
2020-01-18 13:35:58  : month  6  started
2020-01-18 13:35:58  : month  7  started
2020-01-18 13:35:59  : month  8  start

2020-01-18 13:39:46  : month  6  started
2020-01-18 13:39:47  : month  7  started
2020-01-18 13:39:50  : month  8  started
2020-01-18 13:39:53  : month  9  started
2020-01-18 13:39:58  : month  10  started
2020-01-18 13:40:05  : month  11  started
2020-01-18 13:40:13  : month  12  started
Year  2020  is beginning
2020-01-18 13:40:16  : month  1  started
2020-01-18 13:40:17  : month  2  started
2020-01-18 13:40:19  : month  3  started
2020-01-18 13:40:19  : month  4  started
2020-01-18 13:40:20  : month  5  started
2020-01-18 13:40:20  : month  6  started
2020-01-18 13:40:22  : month  7  started
2020-01-18 13:40:23  : month  8  started
2020-01-18 13:40:24  : month  9  started
2020-01-18 13:40:25  : month  10  started
2020-01-18 13:40:26  : month  11  started
2020-01-18 13:40:27  : month  12  started
Loading of games from user  Ajith1411  started
Year  2013  is beginning
2020-01-18 13:40:28  : month  1  started
2020-01-18 13:40:30  : month  2  started
2020-01-18 13:40:31  : month  3  sta

...

2020-01-18 18:41:45  : month  11  started
2020-01-18 18:41:46  : month  12  started
Loading of games from user  motrcolt1  started
Year  2013  is beginning
2020-01-18 18:41:46  : month  1  started
2020-01-18 18:41:46  : month  2  started
2020-01-18 18:41:47  : month  3  started
2020-01-18 18:41:48  : month  4  started
2020-01-18 18:41:49  : month  5  started
2020-01-18 18:41:49  : month  6  started
2020-01-18 18:41:50  : month  7  started
2020-01-18 18:41:50  : month  8  started
2020-01-18 18:41:51  : month  9  started
2020-01-18 18:41:51  : month  10  started
2020-01-18 18:41:51  : month  11  started
2020-01-18 18:41:52  : month  12  started
Year  2014  is beginning
2020-01-18 18:41:52  : month  1  started
2020-01-18 18:41:52  : month  2  started
2020-01-18 18:41:53  : month  3  started
2020-01-18 18:41:54  : month  4  started
2020-01-18 18:41:54  : month  5  started
2020-01-18 18:41:55  : month  6  started
2020-01-18 18:41:55  : month  7  started
2020-01-18 18:41:56  : month  8  star

2020-01-18 18:49:11  : month  6  started
2020-01-18 18:49:12  : month  7  started
2020-01-18 18:49:12  : month  8  started
2020-01-18 18:49:12  : month  9  started
2020-01-18 18:49:13  : month  10  started
2020-01-18 18:49:13  : month  11  started
2020-01-18 18:49:14  : month  12  started
Loading of games from user  Mahbub5  started
Year  2013  is beginning
2020-01-18 18:49:14  : month  1  started
2020-01-18 18:49:15  : month  2  started
2020-01-18 18:49:17  : month  3  started
2020-01-18 18:49:17  : month  4  started
2020-01-18 18:49:18  : month  5  started
2020-01-18 18:49:18  : month  6  started
2020-01-18 18:49:19  : month  7  started
2020-01-18 18:49:20  : month  8  started
2020-01-18 18:49:20  : month  9  started
2020-01-18 18:49:21  : month  10  started
2020-01-18 18:49:22  : month  11  started
2020-01-18 18:49:23  : month  12  started
Year  2014  is beginning
2020-01-18 18:49:23  : month  1  started
2020-01-18 18:49:25  : month  2  started
2020-01-18 18:49:26  : month  3  start

2020-01-18 19:00:47  : month  2  started
2020-01-18 19:00:48  : month  3  started
2020-01-18 19:00:48  : month  4  started
2020-01-18 19:00:48  : month  5  started
2020-01-18 19:00:49  : month  6  started
2020-01-18 19:00:49  : month  7  started
2020-01-18 19:00:49  : month  8  started
2020-01-18 19:00:50  : month  9  started
2020-01-18 19:00:51  : month  10  started
2020-01-18 19:00:52  : month  11  started
2020-01-18 19:00:52  : month  12  started
Loading of games from user  iqbalmir  started
Year  2013  is beginning
2020-01-18 19:00:52  : month  1  started
2020-01-18 19:00:53  : month  2  started
2020-01-18 19:00:53  : month  3  started
2020-01-18 19:00:54  : month  4  started
2020-01-18 19:00:54  : month  5  started
2020-01-18 19:00:54  : month  6  started
2020-01-18 19:00:55  : month  7  started
2020-01-18 19:00:55  : month  8  started
2020-01-18 19:00:55  : month  9  started
2020-01-18 19:00:56  : month  10  started
2020-01-18 19:00:57  : month  11  started
2020-01-18 19:00:57  :

2020-01-18 19:12:59  : month  10  started
2020-01-18 19:12:59  : month  11  started
2020-01-18 19:13:00  : month  12  started
Year  2020  is beginning
2020-01-18 19:13:01  : month  1  started
2020-01-18 19:13:02  : month  2  started
2020-01-18 19:13:03  : month  3  started
2020-01-18 19:13:04  : month  4  started
2020-01-18 19:13:05  : month  5  started
2020-01-18 19:13:05  : month  6  started
2020-01-18 19:13:06  : month  7  started
2020-01-18 19:13:07  : month  8  started
2020-01-18 19:13:07  : month  9  started
2020-01-18 19:13:08  : month  10  started
2020-01-18 19:13:09  : month  11  started
2020-01-18 19:13:10  : month  12  started
Loading of games from user  krautrock  started
Year  2013  is beginning
2020-01-18 19:13:11  : month  1  started
2020-01-18 19:13:12  : month  2  started
2020-01-18 19:13:13  : month  3  started
2020-01-18 19:13:13  : month  4  started
2020-01-18 19:13:13  : month  5  started
2020-01-18 19:13:14  : month  6  started
2020-01-18 19:13:15  : month  7  sta

2020-01-18 19:26:44  : month  5  started
2020-01-18 19:26:50  : month  6  started
2020-01-18 19:27:00  : month  7  started
2020-01-18 19:27:09  : month  8  started
2020-01-18 19:27:22  : month  9  started
2020-01-18 19:27:35  : month  10  started
2020-01-18 19:27:50  : month  11  started
2020-01-18 19:28:01  : month  12  started
Year  2020  is beginning
2020-01-18 19:28:14  : month  1  started
2020-01-18 19:28:20  : month  2  started
2020-01-18 19:28:20  : month  3  started
2020-01-18 19:28:20  : month  4  started
2020-01-18 19:28:21  : month  5  started
2020-01-18 19:28:21  : month  6  started
2020-01-18 19:28:22  : month  7  started
2020-01-18 19:28:22  : month  8  started
2020-01-18 19:28:22  : month  9  started
2020-01-18 19:28:23  : month  10  started
2020-01-18 19:28:24  : month  11  started
2020-01-18 19:28:24  : month  12  started
Loading of games from user  dant0l  started
Year  2013  is beginning
2020-01-18 19:28:24  : month  1  started
2020-01-18 19:28:25  : month  2  starte

In [50]:
load_games('data/Steinstone_2018_06.json')

/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/roman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url
0,chess,bullet,60,True,1,0-1,2018-06-17,13:48:39,2018-6,Steinstone,StevenB7,1424,1448,timeout,win,0.0,1.0,0.465516,0.534484,C00,https://www.chess.com/openings/French-Defense-Normal-Variation,https://www.chess.com/live/game/2883461095,https://api.chess.com/pub/player/steinstone,https://api.chess.com/pub/player/stevenb7
0,chess,bullet,60,True,1,1-0,2018-06-17,13:51:22,2018-6,paljuv2017,Steinstone,1418,1401,win,resigned,1.0,0.0,0.524445,0.475555,B10,https://www.chess.com/openings/Caro-Kann-Defense-2.Nc3-d5,https://www.chess.com/live/game/2883465157,https://api.chess.com/pub/player/paljuv2017,https://api.chess.com/pub/player/steinstone
0,chess,blitz,300,True,1,1-0,2018-06-18,11:31:17,2018-6,Steinstone,Muhmma00,1569,1438,win,timeout,1.0,0.0,0.680071,0.319929,A43,https://www.chess.com/openings/Old-Benoni-Defense,https://www.chess.com/live/game/2885291576,https://api.chess.com/pub/player/steinstone,https://api.chess.com/pub/player/muhmma00
0,chess,blitz,300,True,1,1-0,2018-06-18,11:42:16,2018-6,kevin0890,Steinstone,1567,1542,win,resigned,1.0,0.0,0.535916,0.464084,D02,https://www.chess.com/openings/Queens-Pawn-Opening-Zukertort-Variation,https://www.chess.com/live/game/2885308455,https://api.chess.com/pub/player/kevin0890,https://api.chess.com/pub/player/steinstone
0,chess,blitz,300,True,1,0-1,2018-06-18,12:14:51,2018-6,Rubawa,Steinstone,1449,1561,timeout,win,0.0,1.0,0.344179,0.655821,D00,https://www.chess.com/openings/Queens-Pawn-Opening-Accelerated-London-System,https://www.chess.com/live/game/2885356413,https://api.chess.com/pub/player/rubawa,https://api.chess.com/pub/player/steinstone
0,chess,blitz,300,True,1,0-1,2018-06-18,12:52:55,2018-6,Steinstone,Yovanpeterboy,1538,1559,resigned,win,0.0,1.0,0.469815,0.530185,B06,https://www.chess.com/openings/Modern-Defense-with-1-e4-2.d4,https://www.chess.com/live/game/2885414425,https://api.chess.com/pub/player/steinstone,https://api.chess.com/pub/player/yovanpeterboy
0,chess,blitz,300,True,1,1-0,2018-06-18,12:58:16,2018-6,engski,Steinstone,1540,1516,win,resigned,1.0,0.0,0.534484,0.465516,B12,https://www.chess.com/openings/Caro-Kann-Defense-Advance-Botvinnik-Carls-Defense,https://www.chess.com/live/game/2885423308,https://api.chess.com/pub/player/engski,https://api.chess.com/pub/player/steinstone
0,chess,bullet,60,True,1,0-1,2018-06-22,20:53:44,2018-6,F4lC0nsill0696,Steinstone,1431,1425,checkmated,win,0.0,1.0,0.508634,0.491366,B10,https://www.chess.com/openings/Caro-Kann-Defense-2.Nf3-d5,https://www.chess.com/live/game/2894652172,https://api.chess.com/pub/player/f4lc0nsill0696,https://api.chess.com/pub/player/steinstone
0,chess,bullet,60,True,1,1-0,2018-06-22,20:55:00,2018-6,Steinstone,lithis,1448,1466,win,checkmated,1.0,0.0,0.474119,0.525881,B12,https://www.chess.com/openings/Caro-Kann-Defense-2.d4,https://www.chess.com/live/game/2894654362,https://api.chess.com/pub/player/steinstone,https://api.chess.com/pub/player/lithis
0,chess,bullet,60,True,1,1-0,2018-06-22,20:57:44,2018-6,Steinstone,Bobbysherin11,1464,1375,win,timeout,1.0,0.0,0.625351,0.374649,B00,https://www.chess.com/openings/Kings-Pawn-Opening-Owen-Defense-2.d4,https://www.chess.com/live/game/2894658697,https://api.chess.com/pub/player/steinstone,https://api.chess.com/pub/player/bobbysherin11


In [ ]:
# collect players by countries
# countries_list=['RU']
# get_county_players_list(countries_list[0])

# user_list=pd.read_json('player_lists/players_list_RU_2019-08-25 10:47:56.json')['players'].values
# user_list=['andreyvict', 'sever043', 'Rosolimo']